In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

In [2]:
df = pd.read_csv('/content/drive/MyDrive/Machine Learning 23 24/Data/train (1).csv')

In [3]:
df

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,15694510,Hsueh,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,15741417,Kao,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,15766172,Chiemenam,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165029,165029,15667085,Meng,667,Spain,Female,33.0,2,0.00,1,1.0,1.0,131834.75,0
165030,165030,15665521,Okechukwu,792,France,Male,35.0,3,0.00,1,0.0,0.0,131834.45,0
165031,165031,15664752,Hsia,565,France,Male,31.0,5,0.00,1,1.0,1.0,127429.56,0
165032,165032,15689614,Hsiung,554,Spain,Female,30.0,7,161533.00,1,0.0,1.0,71173.03,0


In [4]:
df.columns

Index(['id', 'CustomerId', 'Surname', 'CreditScore', 'Geography', 'Gender',
       'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [5]:
df.dtypes

id                   int64
CustomerId           int64
Surname             object
CreditScore          int64
Geography           object
Gender              object
Age                float64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard          float64
IsActiveMember     float64
EstimatedSalary    float64
Exited               int64
dtype: object

In [6]:
df.isna().sum()

id                 0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [7]:
cols = ['CreditScore', 'Age', 'Balance', 'EstimatedSalary', 'HasCrCard', 'IsActiveMember', 'Tenure', 'Geography', 'Gender']

In [8]:
X = df[cols]
y = df['Exited']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
ohe = OneHotEncoder()
ct = make_column_transformer(
    (ohe, ['Geography', 'Gender']),
     remainder = 'passthrough')

In [10]:
xgb_model = xgb.XGBClassifier(
    learning_rate=.01,
    n_estimators=2100,
    max_depth=2,
    tree_method = 'exact',
    eval_metric = 'logloss',
)

pipe = make_pipeline(ct, xgb_model)
pipe.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Geography', 'Gender'])])),
                ('xgbclassifier',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, device=None,
                               early_stopping_rounds=None,
                               enable_categoric...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.01,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=2, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=2100, n_jobs=None,
                               num_parallel_tree=None, random_state=None, ...))])

In [11]:
y_pred = pipe.predict(X_test)
accuracy_xg = accuracy_score(y_test, y_pred)
print('Accuracy: ' + str(accuracy_xg))

Accuracy: 0.8385191020086649


In [13]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=2)
scores = cross_val_score(pipe, X, y, cv=cv, scoring='accuracy')

# Print the cross-validation scores
print("Cross-Validation Scores:", scores)
print("Mean Accuracy:", np.mean(scores))

Cross-Validation Scores: [0.83788287 0.83864029 0.83785258 0.83912503 0.83802945]
Mean Accuracy: 0.8383060443508903


In [14]:
df_test = pd.read_csv('/content/drive/MyDrive/Machine Learning 23 24/Data/test (1).csv')

In [15]:
X_new = df_test[cols]

In [16]:
final_pred = pipe.predict_proba(X_new)

In [18]:
final_pred

array([[0.9244741 , 0.07552591],
       [0.3960517 , 0.6039483 ],
       [0.88991976, 0.11008027],
       ...,
       [0.9370905 , 0.06290948],
       [0.9510464 , 0.04895359],
       [0.8443729 , 0.15562706]], dtype=float32)

In [26]:
exited_pred = []
for i in final_pred:
  exited_pred.append(i[1])

In [27]:
kaggle = pd.DataFrame(data = {'id':df_test['id'], 'Exited':exited_pred})
kaggle.set_index('id', inplace=True)

In [29]:
kaggle.to_csv('/content/drive/MyDrive/Machine Learning 23 24/Data/bank_predictions_1.csv')